In [1]:
import os
import pandas as pd

from dfg_rating.model.rating.elo_rating import ELORating
from dfg_rating.model.network.base_network import WhiteNetwork
from dfg_rating.model.forecast.true_forecast import LogFunctionForecast
from dfg_rating.model.rating.controlled_trend_rating import ControlledTrendRating, ControlledRandomFunction


In [2]:
data_football_national = pd.read_csv(os.path.join('..', '..', '..', 'data', 'real', 'Data_Football_National.csv'))
#data_football_national = pd.read_csv("C:/Users/Dell-PC/Documents/Projekte/dfg-rating/data/real/Data_Football_National.csv")

In [3]:
football_network = WhiteNetwork(
    data=data_football_national,
    #node1 = away
    mapping={
        "node1": {
            "id": "AwayID",
            "name": "AwayTeam",
        },
        "node2": {
            "id": "HomeID",
            "name": "HomeTeam",
        },
        "day": "Date",
        "dayIsTimestamp": True,
        "ts_format": "%d.%m.%Y",
        "season": "Season",
        "result": "ResultFT",
        "round": "day",
        "odds": {
            "maximumodds": {
                "home": "OddsHomeMax",
                "draw": "OddsDrawMax",
                "away": "OddsAwayMax"
            },
            "averageodds": {
                "home": "OddsHomeAvg",
                "draw": "OddsDrawAvg",
                "away": "OddsAwayAvg"
            },       
        },
        "bets": {}
    }
    
)

Network loaded correctly


In [4]:
#def first_season(edge):
#    return (edge[2]["season"] == 2010)
football_network.print_data(schedule=True)

Network schedule
(278 vs. 259 at season 2007/2008 round 0, day 0)
(34 vs. 135 at season 2008/2009 round 1, day 1)
(46 vs. 309 at season 2009/2010 round 1, day 1)
---------------


In [5]:
football_network.add_rating(
    rating=ControlledTrendRating(
        starting_point=ControlledRandomFunction(distribution='normal', loc=1000, scale=100),
        delta=ControlledRandomFunction(distribution='normal', loc=0, scale=3),
        trend=ControlledRandomFunction(distribution='normal', loc=0, scale=20/365),
        season_delta=ControlledRandomFunction(distribution='normal', loc=0, scale=10)),
    rating_name='true_rating'
)

football_network.add_forecast(
    forecast=LogFunctionForecast(outcomes=['home', 'draw', 'away'], coefficients=[-1.2, 0.0], beta_parameter=0.006),
    forecast_name='bookmaker_forecast',
    base_ranking='true_rating'
)

#football_network.add_rating(
#    new_rating=ELORating(trained=True, **{'param.c' : 10,
#                                           'param.d' : 400,
#                                           'param.k' : 14,
#                                           'param.w' : 50}),
#                                            rating_name='player_rating'
#    )
#football_network.data.nodes[278]

#football_network.export(ratings = ['player_rating'])

#football_data = pd.read_excel(os.path.join('..', '..', 'data', 'real', 'nation_teams_competitions.xlsx'), engine='openpyxl')

In [13]:
list(football_network.data.edges)

[(259, 278, 0), (135, 34, 0), (309, 46, 0)]

In [14]:
football_network.data.edges[(135, 34, 0)]['forecasts']['bookmaker_forecast'].print()

' h: 0.74 - d: 0.16 - a: 0.10 '

In [ ]:
football_data.Season.unique()

In [ ]:
football_network = WhiteNetwork(
    data=football_data,
    mapping={
        "node1": {
            "id": "Team away",
            "name": "Team away"
        },
        "node2": {
            "id": "Team home",
            "name": "Team home"
        },
        "day": "Date",
        "dayIsTimestamp": True,
        "round": "Round",
        "season": "Season",
        "forecasts": {},
        "odds": {},
        "bets": {}
    }
)

In [ ]:
def first_season(edge):
    return (edge[2]["season"] == "2019/2020")
football_network.print_data(schedule=True, edge_filter=first_season)